In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.extmath import randomized_svd
from sklearn.feature_extraction.text import  TfidfVectorizer
import pickle

In [45]:
platforms = pd.read_csv('DataPlataformas.csv',sep=";")

In [46]:
platforms.head(2)

,id,type,title,director,cast,country,date_added,release_year,listed_in,description,rating,plataforma,scored,userId,duration_int,duration_type
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,"comedy, drama",a small fishing village must procure a local d...,G,amazon,3.467131,99178,113.0,min
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,"drama, international",a metro family decides to fight a cyber crimin...,13+,amazon,3.548682,93837,110.0,min


In [47]:
MachinePeli = platforms[['id', 'title',"userId",'scored']] 
#user_item.reset_index(drop=True)
MachinePeli = MachinePeli

In [50]:
MachinePeli

,id,title,userId,scored
0,as1,the grand seduction,99178,3.467131
1,as2,take care good night,93837,3.548682
2,as3,secrets of deception,84095,3.500000
3,as4,pink: staying true,88177,3.538055
4,as5,monster maker,83381,3.478992
...,...,...,...,...
22993,ns8803,zodiac,87794,3.438998
22994,ns8804,zombie dumb,92085,3.515947
22995,ns8805,zombieland,89944,3.420945
22996,ns8806,zoom,91428,3.588050


In [49]:
#MachinePeli.to_csv('user_item.csv', index=False)

In [51]:
# Vectorizador TfidfVectorizer con parámetros de reduccion procesamiento
vectorizer = TfidfVectorizer(min_df=10, max_df=0.5, ngram_range=(1,2))

# Ajustar y transformar el texto de la columna "descripcion" del DataFrame
X = vectorizer.fit_transform(MachinePeli['title'])

# Calcular la matriz de similitud de coseno con una matriz reducida de 5500x5500
similarity_matrix = cosine_similarity(X[:5500,:])

# Obtener la descomposición en valores singulares aleatoria de la matriz de similitud de coseno con 10 componentes
n_components = 10
U, Sigma, VT = randomized_svd(similarity_matrix, n_components=n_components)

# Construir la matriz reducida de similitud de coseno
reduced_similarity_matrix = U.dot(np.diag(Sigma)).dot(VT)

In [37]:
reduced_similarity_matrix

array([[ 0.08641783,  0.00071048,  0.00077974, ...,  0.00113352,
        -0.00040896,  0.00123587],
       [ 0.00071053,  0.00041597,  0.0027472 , ...,  0.00108938,
         0.00119177,  0.0007825 ],
       [ 0.00077973,  0.00274729,  0.12071956, ...,  0.00820629,
         0.00055155, -0.00072764],
       ...,
       [ 0.00113321,  0.00109173,  0.00820559, ...,  0.0031583 ,
         0.00358696,  0.00207965],
       [-0.00040869,  0.00119161,  0.00055159, ...,  0.00357986,
         0.0064013 ,  0.0027539 ],
       [ 0.0012361 ,  0.00078245, -0.00072764, ...,  0.0020734 ,
         0.00275437,  0.00266105]])

In [52]:
reduced_similarity_df = pd.DataFrame(reduced_similarity_matrix)
reduced_similarity_df.to_csv('MachinePeli.csv', index=False)

In [53]:
reduced_similarity_df

,0,1,2,3,4,5,6,7,8,9,...,5490,5491,5492,5493,5494,5495,5496,5497,5498,5499
0,0.086418,0.000710,0.000780,0.000164,0.000244,-0.000694,8.972459e-19,-0.000036,-9.883877e-20,-0.000173,...,-0.000039,0.000265,-0.000238,0.106893,0.062186,0.0,0.045817,0.001134,-0.000409,0.001236
1,0.000710,0.000416,0.002747,0.000061,0.000128,0.007059,-5.342492e-18,0.000643,4.537519e-19,0.000434,...,0.000159,0.002232,0.001083,0.003457,0.003095,0.0,0.001852,0.001086,0.001195,0.000785
2,0.000780,0.002747,0.120720,0.001946,0.000295,-0.001033,7.613436e-18,-0.000504,-1.030906e-20,0.000451,...,-0.000021,0.086685,0.004626,0.120391,0.107681,0.0,0.000464,0.008206,0.000549,-0.000730
3,0.000164,0.000062,0.001946,0.000034,0.000007,0.000394,-3.657657e-19,0.000014,2.021773e-20,0.000039,...,0.000011,0.001409,0.000138,0.002118,0.001842,0.0,-0.000036,0.000186,0.000105,0.000036
4,0.000244,0.000128,0.000295,0.000007,0.000089,0.001464,-3.440765e-18,0.000088,1.037025e-19,0.000102,...,0.000035,0.000300,0.000159,0.000555,0.000467,0.0,-0.000128,0.000281,0.000204,0.000158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
5496,0.045817,0.001853,0.000464,-0.000036,-0.000128,-0.006092,1.053573e-17,0.012129,-4.228877e-19,-0.000257,...,0.000006,0.003358,0.005392,0.055182,0.039671,0.0,0.173094,0.004398,0.006326,0.000918
5497,0.001134,0.001086,0.008206,0.000185,0.000281,0.019508,-1.683683e-17,0.001696,1.055012e-18,0.001263,...,0.000435,0.006607,0.003061,0.009137,0.008492,0.0,0.004397,0.003134,0.003573,0.002068
5498,-0.000409,0.001193,0.000549,0.000104,0.000204,0.024775,-2.857379e-17,0.002434,1.184731e-18,0.001967,...,0.000664,0.001471,0.004368,-0.000683,0.000724,0.0,0.006326,0.003569,0.006407,0.002757


In [54]:
def get_recommendation(titulo: str):
    try:
        #Ubicamos el indice del titulo pasado como parametro en la columna 'title' del dts user_item
        indice = np.where(MachinePeli['title'] == titulo)[0][0]
        #Encontramos los indices de las puntuaciones y caracteristicas similares del titulo 
        puntuaciones_similitud = reduced_similarity_matrix[indice,:]
        #Ordenamos los indices de menor a mayor
        puntuacion_ordenada = np.argsort(puntuaciones_similitud)[::-1]
        #seleccionamos solo 5 
        top_indices = puntuacion_ordenada[:5]
        #retornamos los 5 items con sus titulos como una lista
        return MachinePeli.loc[top_indices, 'title'].tolist()
        #Si el titulo dado no se encuentra damos un aviso
    except IndexError:
        print(f"El título '{titulo}' no se encuentra en la base de datos. Intente con otro título.")

In [57]:
get_recommendation('the grand seduction')

['the triangle',
 'the retrievers',
 'the swap',
 'the surgeon',
 'bloodrayne: the third reich']